<a href="https://colab.research.google.com/github/abhishekdhankar95/Algo/blob/master/sentence_to_CUI_to_ICD10CM_conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Hi Heather!

##* Following is a short Demo of how ailments could be detected from encounter notes and converted into corresponding ICD codes on a CPU (no need of a GPU).

##* Please scroll down to section 1 below

In [ ]:
!pip install scispacy

In [ ]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_sm-0.5.4.tar.gz

In [ ]:
'''
import spacy
import scispacy

from scispacy.linking import EntityLinker

nlp = spacy.load("en_core_sci_sm")

# This line takes a while, because we have to download ~1GB of data
# and load a large JSON file (the knowledge base). Be patient!
# Thankfully it should be faster after the first time you use it, because
# the downloads are cached.
# NOTE: The resolve_abbreviations parameter is optional, and requires that
# the AbbreviationDetector pipe has already been added to the pipeline. Adding
# the AbbreviationDetector pipe and setting resolve_abbreviations to True means
# that linking will only be performed on the long form of abbreviations.
nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls"})

doc = nlp("Spinal and bulbar muscular atrophy (SBMA) is an \
           inherited motor neuron disease caused by the expansion \
           of a polyglutamine tract within the androgen receptor (AR). \
           SBMA can be caused by this easily.")

# Let's look at a random entity!
entity = doc.ents[1]

print("Name: ", entity)


# Each entity is linked to UMLS with a score
# (currently just char-3gram matching).
print("UMLS entities in order of Relevance: ")
linker = nlp.get_pipe("scispacy_linker")
idx = 0
for umls_ent in entity._.kb_ents:
  idx += 1
  print(str(idx)+" .")
  print(linker.kb.cui_to_entity[umls_ent[0]])
  print("****************************")
'''

In [ ]:
!pip install stanza

# Section 1: Detecting Problems in the Patient's Free Text records

In [ ]:
#This is the encounter note
encounter_note = 'The patient had a sore throat and was treated with Cepacol lozenges.'


import stanza
stanza.download('en', package='mimic', processors={'ner': 'i2b2'})
nlp = stanza.Pipeline('en', package='mimic', processors={'ner': 'i2b2'})



In [33]:
# annotate clinical text
doc = nlp(encounter_note)
# print out all entities
print("Following is the list of problemns and treatments: ")
print()
for ent in doc.entities:
    print(f'{ent.text}\t{ent.type}')

Following is the list of problemns and treatments: 

a sore throat	PROBLEM
Cepacol lozenges	TREATMENT


* **Sore Throat was identified as a problem by the model above**


# Now you may scroll down to the Section 2 below

In [ ]:
#!pip install pymedtermino
#!wget https://download.nlm.nih.gov/umls/kss/IHTSDO2024/IHTSDO20240401/SnomedCT_InternationalRF2_PRODUCTION_20240401T120000Z.zip?_gl=1*dnmbf2*_ga*MzI5Mjk5NTgyLjE3MDk5MTQ0MjY.*_ga_7147EPK006*MTcxMjU5NDAzOC4xMy4xLjE3MTI1OTQwNjEuMC4wLjA.*_ga_P1FPTH9PL4*MTcxMjU5NDAzOC4xMy4xLjE3MTI1OTQwNjEuMC4wLjA.
#!wget https://download.nlm.nih.gov/umls/kss/SNOMEDCT_CORE_SUBSET/SNOMEDCT_CORE_SUBSET_202311.zip?_gl=1*sukimt*_ga*MzI5Mjk5NTgyLjE3MDk5MTQ0MjY.*_ga_7147EPK006*MTcxMjU5NDAzOC4xMy4xLjE3MTI1OTQ3MjkuMC4wLjA.*_ga_P1FPTH9PL4*MTcxMjU5NDAzOC4xMy4xLjE3MTI1OTQ3MjkuMC4wLjA.
!unzip  /content/SnomedCT_InternationalRF2_PRODUCTION_20240401T120000Z.zip
!unzip /content/SNOMEDCT_CORE_SUBSET_202311.zip
#SNOMEDCT_DIR = "/home/jiba/telechargements/base_med/SnomedCT_Release_INT_20130731"
#SNOMEDCT_CORE_FILE = "/home/jiba/telechargements/base_med/SNOMEDCT_CORE_SUBSET_201308.txt"

In [1]:
SNOMEDCT_DIR = "/content/SnomedCT_InternationalRF2_PRODUCTION_20240401T120000Z"
SNOMEDCT_CORE_FILE = "/content/SNOMEDCT_CORE_SUBSET_202311.txt"

In [13]:
!git clone https://github.com/MedevaKnowledgeSystems/pymedtermino.git

Cloning into 'pymedtermino'...
remote: Enumerating objects: 92, done.
remote: Total 92 (delta 0), reused 0 (delta 0), pack-reused 92
Receiving objects: 100% (92/92), 21.24 MiB | 26.36 MiB/s, done.
Resolving deltas: 100% (29/29), done.


In [6]:

!cd pymedtermino && python /content/pymedtermino/setup.py build

/usr/local/lib/python3.10/dist-packages/setuptools/dist.py:755: SetuptoolsDeprecationWarning: Invalid dash-separated options
!!

        ********************************************************************************
        Usage of dash-separated 'force-manifest' will not be supported in future
        versions. Please use the underscore name 'force_manifest' instead.

        This deprecation is overdue, please update your project and remove deprecated
        calls to avoid build errors in the future.

        See https://setuptools.pypa.io/en/latest/userguide/declarative_config.html for details.
        ********************************************************************************

!!
  opt = self.warn_dash_deprecation(opt, section)
/usr/local/lib/python3.10/dist-packages/setuptools/dist.py:755: SetuptoolsDeprecationWarning: Invalid dash-separated options
!!

        ********************************************************************************
        Usage of dash-separated

In [7]:
import pymedtermino

In [8]:
from pymedtermino import *
from pymedtermino.snomedct import *

# Section 2: Converting extracted ailment/problem phrases into SNOMEDCT

In [12]:
# searching for SNOMED_CT equivalents for the PROBLEM(S) identified earlier
SNOMEDCT.search("a sore throat")

[SNOMEDCT[139658009]  # Has a sore throat,
 SNOMEDCT[162388002]  # Has a sore throat (finding)]

After this both the SNOMEDCT alternatives (139658009 & 162388002) can be fed into a system to convert SNOMEDCT to ICD10.
There are two such systems:
* https://github.com/Vicomtech/ClinIDMap
* https://imagic.nlm.nih.gov/imagic/code/map


Both these approaches can be automated.